In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] Transmission</font>
# @markdown > Transmission Default Credential</br>Username: admin</br>Password: admin
!cp /0A/v2r.py /.ipython/v2r.py
USE_FREE_TOKEN = True
TOKEN = ""
REGION = "US"
Tunneling = "argo_tunnel_(cloudflare)" #@param ["argo_tunnel_(cloudflare)", "localhost.run", "ngrok"]

if Tunneling == "argo_tunnel_(cloudflare)":
    PORT_FORWARD = "argotunnel"
elif Tunneling == "localhost.run":
    PORT_FORWARD = "localhost"
elif Tunneling == "ngrok":
    PORT_FORWARD = "ngrok"
# ================================================================ #

import os, psutil, time, urllib.request, pathlib
from IPython.display import HTML, clear_output

HOME = os.path.expanduser("~")

from v2r import (
    checkAvailable,
    loadingAn,
    PortForward_wrapper,
    runSh,
    displayUrl,
    findProcess
)

loadingAn()

if not os.path.exists('/usr/bin/transmission-daemon'):
    os.makedirs('downloads', exist_ok=True)
    os.makedirs('tools/transmission/', exist_ok=True)
    runSh('apt install transmission-daemon')
    nTWC = "https://raw.githubusercontent.com/ronggang/" \
        "transmission-web-control/master/release/install-tr-control.sh"
    urllib.request.urlretrieve(nTWC, 'tools/transmission/trInstall.sh')
    runSh('bash tools/transmission/trInstall.sh auto')
    
    try:
        pathlib.Path('tools/transmission/trInstall.sh').unlink()
    except FileNotFoundError:
        pass

if not findProcess('transmission-daemon', '--no-watch-dir'):
    !transmission-daemon --no-watch-dir --config-dir tools/transmission \
    --port 9091 --download-dir /content/downloads/ --dht --utp --no-portmap \
    --peerlimit-global 9999 --peerlimit-torrent 9999 --no-global-seedratio \
    -u admin -v admin --auth

clear_output()

server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['vuze', 9595, 'http'], ['transmission', 9091, 'http']], REGION.lower, [f"{HOME}/.ngrok2/transmission.yml", 4058]).start('transmission', displayB=False)
displayUrl(server, pNamU='Transmission : ', btc='r')